# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [2]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
pd.read_sql("""
SELECT customerNumber,
       contactLastName,
       contactFirstName
FROM customers
WHERE customerNumber IN (SELECT customerNumber
                         FROM orders
                         WHERE orderDate = '2003-01-31');
""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [12]:
# Your code here
pd.read_sql("""
SELECT COUNT(orderNumber) AS num_ordered,
       SUM(quantityOrdered) AS num_items,
       productName
FROM orderdetails
JOIN products
     USING (productCode)
GROUP BY productName
ORDER BY num_items DESC;
""", conn)

,num_ordered,num_items,productName
0,53,1808,1992 Ferrari 360 Spider red
1,28,1111,1937 Lincoln Berline
2,28,1085,American Airlines: MD-11S
3,28,1076,1941 Chevrolet Special Deluxe Cabriolet
4,28,1074,1930 Buick Marquette Phaeton
...,...,...,...
104,25,855,1999 Indy 500 Monte Carlo SS
105,25,832,1911 Ford Town Car
106,25,824,1936 Mercedes Benz 500k Roadster
107,25,803,1970 Chevy Chevelle SS 454


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [15]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql("""
SELECT productName,
       COUNT(DISTINCT customerNumber) AS num_people
FROM products
JOIN orderdetails
      USING(productCode)
JOIN orders
      USING(orderNumber)
GROUP BY productName
ORDER BY num_people DESC;
""", conn)

,productName,num_people
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [22]:
# Your code here
pd.read_sql("""
SELECT
    DISTINCT employeeNumber,
    officeCode,
    o.city,
    firstName,
    lastName
FROM employees
JOIN offices AS o
    USING(officeCode)
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
WHERE productCode IN (SELECT productCode
                      FROM products
                      JOIN orderdetails
                          USING(productCode)
                      JOIN orders
                          USING(orderNumber)
                      GROUP BY productCode
                      HAVING COUNT(DISTINCT customerNumber) < 20);
""", conn)

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [30]:
# Your code here
pd.read_sql("""
SELECT
    DISTINCT employeeNumber,
    firstName,
    lastName,
    COUNT(DISTINCT customerNumber) AS num_of_customers,
    CAST(creditLimit AS int) AS "credit_limit"
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING credit_limit > 15000;
""", conn)

,employeeNumber,firstName,lastName,num_of_customers,credit_limit
0,1165,Leslie,Jennings,6,210500
1,1166,Leslie,Thompson,6,71800
2,1188,Julie,Firrelli,6,43400
3,1216,Steve,Patterson,6,100600
4,1286,Foon Yue,Tseng,7,138500
5,1323,George,Vanauf,8,114900
6,1337,Loui,Bondur,6,123900
7,1370,Gerard,Hernandez,7,21000
8,1401,Pamela,Castillo,10,83400
9,1501,Larry,Bott,8,96500


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!